<a href="https://colab.research.google.com/github/who-am-i-h/AI-vision/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask
!pip install matplotlib
!pip install tensorflow

In [2]:
from google.colab import files

files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"whoamiishi","key":"a7415d0a49510f7fbd66444b882117f8"}'}

In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/


In [4]:
!chmod 600 ~/.kaggle/kaggle.json


In [5]:
!kaggle datasets download -d abhranta/brain-tumor-detection-mri


Dataset URL: https://www.kaggle.com/datasets/abhranta/brain-tumor-detection-mri
License(s): unknown
 98% 63.0M/64.3M [00:04<00:00, 19.6MB/s]
100% 64.3M/64.3M [00:04<00:00, 14.0MB/s]


In [6]:
import zipfile
import os

with zipfile.ZipFile('/content/brain-tumor-detection-mri.zip', 'r') as zip_ref:
    zip_ref.extractall('brain_data')

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout



In [8]:
base_dir = '/content/brain_data/Brain_Tumor_Detection'
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.15
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(128, 128),
    batch_size=32,
    classes=['yes', 'no'],  # Only include 'yes' and 'no'.
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(128, 128),
    batch_size=32,
    classes=['yes', 'no'],  # Only include 'yes' and 'no'.
    class_mode='binary',
    subset='validation'
)


Found 2550 images belonging to 2 classes.
Found 450 images belonging to 2 classes.


In [9]:
# print(len(os.listdir(base_dir + "/yes")))
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Dropout to reduce overfitting
    Dense(1, activation='sigmoid')  # Binary output
])



/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Binary classification
    metrics=['accuracy']
)


In [11]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,  # Start with 20 epochs
    verbose=1
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


80/80 ━━━━━━━━━━━━━━━━━━━━ 26s 232ms/step - accuracy: 0.5405 - loss: 0.7412 - val_accuracy: 0.6578 - val_loss: 0.6253
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 17s 194ms/step - accuracy: 0.6883 - loss: 0.5883 - val_accuracy: 0.6200 - val_loss: 0.6144
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 16s 183ms/step - accuracy: 0.6837 - loss: 0.5791 - val_accuracy: 0.7156 - val_loss: 0.5710
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 16s 182ms/step - accuracy: 0.7270 - loss: 0.5552 - val_accuracy: 0.7333 - val_loss: 0.5614
Epoch 5/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 16s 182ms/step - accuracy: 0.7506 - loss: 0.5174 - val_accuracy: 0.7400 - val_loss: 0.5679
Epoch 6/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 21s 189ms/step - accuracy: 0.7206 - loss: 0.5507 - val_accuracy: 0.6067 - val_loss: 0.7370
Epoch 7/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 16s 182ms/step - accuracy: 0.7470 - loss: 0.5181 - val_accuracy: 0.7778 - val_loss: 0.4939
Epoch 8/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 16s 184ms/step - accuracy: 0.7720 - loss: 0.4885 - val_accuracy: 0.662

In [14]:

pred_dir = f'{base_dir}/pred'

pred_generator = ImageDataGenerator(rescale=1.0/255).flow_from_directory(
    pred_dir,
    target_size=(128, 128),
    batch_size=1,
    class_mode=None,
    shuffle=False
)


Found 0 images belonging to 0 classes.


In [15]:
model.save('brain_tumor_detection_model.h5')


In [22]:
from tensorflow.keras.preprocessing import image
import numpy as np


def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Example
img_path = '/content/brain_data/Brain_Tumor_Detection/pred/pred5.jpg'
img_array = preprocess_image(img_path)
prediction = model.predict(img_array)
if prediction[0] > 0.5:
    print("Prediction: No Cancer Tumor Detected")
else:
    print("Prediction: Cancer Tumor Detected")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Prediction: Cancer Tumor Detected


In [ ]:
!pip install flask-ngrok